In [1]:
%load_ext autoreload
%autoreload 2

from main_huh import main, get_args
import os

In [22]:

input_ = "--task sine --n_contexts 2 --lrs 0.05 0.05 --n_iters 3 10 --k_batch_train 5 4 --n_batch_train 5 4 --architecture 1 28 1"
args = get_args(input_.split())
# args.k_batch_train = [5, 4]
# args.k_batch_test =  [5, 3]
# args.k_batch_valid = [5, 2]
# args.n_batch_train = [5, 4]
# args.n_batch_test = [5, 3]
# args.n_batch_valid = [5, 2]
# args.for_iters = [ 1, 1 ]
# args.lrs = [ 1.0, 0.01] 
# args.test_interval = 1 
# args.ctx_logging_levels=[0,]

# args.save_path = os.getcwd()

print(args)

main(args)


Namespace(architecture=[1, 28, 1], ctx_logging_levels=[], data_parallel=False, device=device(type='cuda', index=0), encoders=[None, None, None], first_order=False, for_iters=None, higher_flag=False, k_batch_test=None, k_batch_train=[5, 4], k_batch_valid=None, log_interval=100, log_name='experiment', loss_logging_levels=[], lrs=[0.05, 0.05], model_type='CAVIA', n_batch_test=None, n_batch_train=[5, 4], n_batch_valid=None, n_contexts=[2], n_iters=[3, 10], prefix='', save_path='/nobackup/users/benhuh/Projects/cavia/shared_results', seed=42, task='sine', test_interval=0, v_num=None, viz=False)
experiment
/nobackup/users/benhuh/Projects/cavia/shared_results/logs/experiment
max_iters [3, 10]
train [(0.990942339314793, 0.5761822607907845), (0.200864022049432, 3.0470616661964836), (0.864491338167939, 0.49007123908847783), (1.590786990501735, 1.648570950539833)]
test [(2.2165305913463675, 0.9149233273574404), (4.178968939922067, 0.6670829901766624), (1.5315087778225691, 1.1509596754470768), (0.3

ipdb>  cont


optimize/lv1_task/train/lv0_task/train loss_final 0.7870330214500427
optimize/lv1_task/train/lv0_task/train loss_final 1.4502296447753906
optimize/lv1_task/train/lv0_task/train loss_final 0.2600858807563782
optimize/lv1_task/train/lv0_task/train loss_final 2.686105489730835
optimize/lv1_task/train/lv0_task/train loss_final 0.8500445485115051
optimize/lv1_task/train/lv0_task/train loss_final 0.0743284523487091
optimize/lv1_task/train/lv0_task/train loss_final 0.9483712315559387
optimize/lv1_task/train/lv0_task/train loss_final 0.3211040496826172
optimize/lv1_task/train/lv0_task/train loss_final 0.63457852602005
optimize/lv1_task/train/lv0_task/train loss_final 0.18605878949165344
optimize/lv1_task/train/lv0_task/train loss_final 0.3470713794231415
optimize/lv1_task/train/lv0_task/train loss_final 2.037562131881714
optimize/lv1_task/train/lv0_task/train loss_final 0.30934569239616394
optimize/lv1_task/train/lv0_task/train loss_final 0.5465360879898071
optimize/lv1_task/train/lv0_task/tra

In [ ]:
# def sine_gen(params):
# #     return input_1d_gen, partial(sine1_fnc, params)
#     return Task_sampler(task_fnc = partial(sine1_fnc, params), task_params = input_1d_gen, k_batches = k_lv0)

# def line_gen(params):
#     return Task_sampler(task_fnc = partial(line1_fnc, params), task_params = input_1d_gen, k_batches = k_lv0)



In [40]:
from train_huh import get_task
from test import *
from dataset import Hierarchical_Task
from functools import partial

_, batch_dict = get_task(args)


k_lv1 =  [3,2,0]
k_lv0 =  [5,4,0]


###########################
# Level0

def sine1_fnc(params, x):
    amplitude, phase = params
    return np.sin(x - phase) * amplitude

def line1_fnc(params, x):
    slope, bias = params
    return slope * x + bias

def lv0_task(task_fnc, input_fnc, k_lv0):
    def fnc_(params):
        return Task_sampler(task_fnc = partial(task_fnc, params), task_params = input_fnc, k_batches = k_lv0)
    return fnc_

sine0_task = lv0_task(sine1_fnc, input_1d_gen, k_lv0)
line0_task = lv0_task(line1_fnc, input_1d_gen, k_lv0)
###########################
# Level1
sine1_task = Task_sampler(task_fnc = sine0_task, task_params = sine_param_gen, k_batches = k_lv1)
line1_task = Task_sampler(task_fnc = line0_task, task_params = line_param_gen, k_batches = k_lv1)

###########################
# Level2
sine_line2_dict = {'sine': sine1_task, 'line': line1_task}
sine_line2_task = Task_sampler(task_fnc = sine_line2_dict, task_params = None, k_batches = None)

supertask = Hierarchical_Task(sine_line2_task, batch_dict=batch_dict, idx=0)


In [41]:
supertask.loader_dict['train'].dataset[0].loader_dict['train'].dataset[0][1].get_data('train')

(tensor([[-3.3697],
         [ 3.1030],
         [ 1.6098],
         [ 3.9108],
         [-2.7181]]), tensor([[ 1.5345],
         [ 0.9682],
         [ 3.1639],
         [-1.5699],
         [-0.5145]]))

In [31]:


def in2_fnc(batch):
    return random.choice(['sine', 'line'])

in1_fnc = sine_param_gen
in0_fnc = input_1d_gen

# def fn2(params):
#     return {'sine': sine1_fnc, 'line': line1_fnc}[params]

# in2 = in2_fnc(1)
# print(in2)
# out2= fn2(in2)
# fn1 = out2
# print(fn1)

# in1 = in1_fnc(2)
# print(in1)
# fn0 = partial(fn1,in1[0])
# print(fn1)

# in0 = in0_fnc(3)
# out0 = fn0(in0)
# print(out0)

#######################################################

# def fn2(params2, params1, x):
#     return {'sine': sine1_fnc(params1, x), 'line': line1_fnc(params1, x)}[params2]

# in2 = in2_fnc(1)
# print(in2)
# out1= partial(fn2,in2)
# print(out1)
# fn1 = out1

# in1 = in1_fnc(2)
# print(in1)
# fn0 = partial(fn1,in1[0])
# print(fn1)

# in0 = in0_fnc(3)

# print(fn0(in0))

#######################################################

        

def fn2(params2, params1, x):
    return {'sine': (sine1_fnc(params1, x), sine_param_gen), 'line': (line1_fnc(params1, x), line_param_gen)}[params2]

def fn_in2(params2, params1, x):
    fn_ = {'sine': sine1_fnc(params1, x), 'line': line1_fnc(params1, x)}[params2]
    in_ = {'sine': sine_param_gen, 'line': line_param_gen}[params2]
    return fn_, in_

partial(fn_in2, params2) = partial(fn_, params2), partial(fn_, params2), 


in2 = in2_fnc(1)
print(in2)
out1= partial(fn2,in2)
print(out1)
fn1, in1_fnc = out1
# fn2= sine_line2_fnc(in2)
# print(fn2)

# in1_fnc = sine_param_gen
in1 = in1_fnc(2)
print(in1)
fn0 = partial(fn1,in1[0])
print(fn1)

in0 = input_1d_gen(3)

print(fn0(in0))

# # def lv3_fnc():
# #     return sine_line2_fnc, in2_fnc

# # sineline2_task = lv0_task(sine_line2_fnc, in2_fnc, [2,0,0])

# # fn2, in2_fnc = lv3_fnc()

# in2 = in2_fnc(1)
# print(in2)
# out1= partial(fn2,in2)
# print(out1)
# fn1, in1_fnc = out1
# # fn2= sine_line2_fnc(in2)
# # print(fn2)

# # in1_fnc = sine_param_gen
# in1 = in1_fnc(2)
# print(in1)
# fn0 = partial(fn1,in1[0])
# print(fn1)

# in0 = input_1d_gen(3)

# print(fn0(in0))


sine
functools.partial(<function fn2 at 0x7fdea084d560>, 'sine')


TypeError: cannot unpack non-iterable functools.partial object

In [91]:
from functools import partial


def in_fnc2(batch):
    return random.choice(['sine', 'line'])
    

# def out_fnc2(params):
#     dict_ = {'sine': sine_gen ,'line': line_gen}
#     return dict_[params]
    
def out_fnc2(params2):
    
    def in_fnc1(batch):
        return np.random.rand(batch), np.random.rand(batch)
    
    def sine_gen(params1):

        def out_fnc1(params, x):
            amplitude, phase = params
            return np.sin(x - phase) * amplitude

        return in_fnc1, partial(out_fnc1, params1)

    def line_gen(params1):

#         def in_fnc1(params, batch):
#             return np.random.rand(batch), np.random.rand(batch)
        def out_fnc1(params, x):
            slope, bias = params
            return slope * x + bias

        return partial(in_fnc1, params1), partial(out_fnc1, params1)
    
    dict_ = {'sine': sine_gen ,'line': line_gen}
    return in_fnc1, dict_[params2]        
    
    
#     return partial(in_fnc2, params2), partial(out_fnc2, params2)
#     return partial(out_fnc2, params2)
#     return out_fnc2(params2)

###########################



in2 = in_fnc2(1)
print(in2)
in_fnc1, fn2 = out_fnc2(in2)
print(fn2)
print(in_fnc1)
in1 = in_fnc1(1)
print(in1)
in0, fn1 = fn2(in1)


line
<function out_fnc2.<locals>.line_gen at 0x7fc33063a0e0>
<function out_fnc2.<locals>.in_fnc1 at 0x7fc2c001b4d0>
(array([0.05188082]), array([0.82568739]))


In [77]:
p1 = in2(1)
in1, fn1 = fn2(p1)(None)
print(in1)
print(fn1)


TypeError: 'str' object is not callable

In [68]:
p0 = in1(1)
amplitude, phase = p0
print(amplitude, phase)
fn1((amplitude, phase))
# in1, fn1 = fn1(p0)(None)
# print(in1)
# print(fn1)


[0.49798632] [0.68617783]


TypeError: cannot unpack non-iterable NoneType object

In [ ]:

# level 2: label  ->  sine, linear, 
# level 1: coeffs ->  sine(c1), sine(c2), 
# level 0: x      ->  sine(c1)(x1,x2,...), sine(c2)(x1,x2,...) 


lv2_fnc(name='sine')(slope=2, bias=1)(x=0.5)

lv2_fnc(name='sine') ->  lv2_param_gen = ['sine','linear'] + lv2_task = dict_

lv2_task(lv2_param) ->  lv1_param_gen (slope/bias) + lv1_task:  -> 

lv1_task(lv1_param)  ->  lv0_param_gen (x) + lv0_task

lv0_task(lv0_x)      ->  lv0_y

lv0_target = lv0_fnc(lv0_x_gen)

In [20]:
fnc_dict = {'linear': sample_sin_fnc, 'sine': sample_linear_fnc }
print(list(fnc_dict))
# in_ = list(fnc_dict.keys())
print(list(fnc_dict.keys()))
print(list(fnc_dict.values()))

fnc = fnc_dict.get
fnc('linear')

['linear', 'sine']
['linear', 'sine']
[<bound method Task_sampler.sample of <test.Task_sampler object at 0x7fc2c0034390>>, <bound method Task_sampler.sample of <test.Task_sampler object at 0x7fc2c0034290>>]


<bound method Task_sampler.sample of <test.Task_sampler object at 0x7fc2c0034390>>

In [17]:
input_1d_gen(5)

tensor([[-3.8985],
        [ 0.3744],
        [-2.1175],
        [-4.7005],
        [-3.5485]])

In [209]:
tasks1 = next(iter(supertask.load('train')))
tasks0 = next(iter(tasks1[0].load('train')))
print(tasks0)

[tensor([[ 2.1429],
        [-2.1429],
        [-5.0000],
        [ 0.7143],
        [ 5.0000]]), tensor([[ 11.0295],
        [ -6.5984],
        [-18.3504],
        [  5.1535],
        [ 22.7815]])]


In [9]:

import torchvision.datasets as datasets

data_dir    = '/nobackup/users/benhuh/data/' 
train_imgs = datasets.MNIST(data_dir, train=True, transform=[], download=True)

In [32]:
args = get_args("")
args.log_interval=50

args.k_batch_train = [10, 3, 2]
args.k_batch_test =  [10, 3, 2]
args.k_batch_valid = [10, 3, 2]
args.n_batch_train = [10, 3, 2]
args.n_batch_test = [10, 3, 2]
args.n_batch_valid = [10, 3, 2]
args.architecture = [ 1, 28, 1 ]
args.n_iters = [10, 10, 20 ] 
args.for_iters = [ 5, 3, 1 ]
args.task = 'sine+linear' # ['LQR']
args.n_contexts = [ 3, 2 ]
args.lrs = [ 1.0, 0.0, 0.001] 
args.test_interval = 100 
args.classes = [0]

# args.log_name=

args.ctx_logging_levels=[0,]

print(args)

main(args)


Namespace(architecture=[1, 28, 1], classes=[0], ctx_logging_levels=[0], data_parallel=False, device=device(type='cuda', index=0), encoders=[None, None, None], first_order=False, for_iters=[5, 3, 1], higher_flag=False, k_batch_test=[10, 3, 2], k_batch_train=[10, 3, 2], k_batch_valid=[10, 3, 2], load_model='', log_interval=50, log_name=None, lrs=[1.0, 0.0, 0.001], model_type='CAVIA', n_batch_test=[10, 3, 2], n_batch_train=[10, 3, 2], n_batch_valid=[10, 3, 2], n_contexts=[3, 2], n_iters=[10, 10, 20], prefix='', save_path='/nobackup/users/benhuh/Projects/cavia/shared_results', seed=42, task='sine+linear', test_interval=100, v_num=None, viz=False)
sine+linear
/nobackup/users/benhuh/Projects/cavia/shared_results/logs/sine+linear
max_iters [10, 10, 20]
level 2
batch 2
task <bound method Label_pre_sampler.sample of <task.make_tasks_new.Label_pre_sampler object at 0x2000afe3af10>>
> /nobackup/users/benhuh/Projects/cavia/regression/dataset.py(112)__init__()
    110         set_trace()
    111 
-

ipdb>  cont


level 2 type train
> /nobackup/users/benhuh/Projects/cavia/regression/task/make_tasks_new.py(31)sample()
     29     def sample(self, sample_type):
     30         set_trace()
---> 31         return self.label_dict[sample_type]
     32 
     33 



ipdb>  self.label_list


[<function sample_sin_fnc at 0x20000b76eef0>, <function sample_linear_fnc at 0x2000afd1a680>]


ipdb>  self.label_dict


{'train': [<function sample_sin_fnc at 0x20000b76eef0>, <function sample_linear_fnc at 0x2000afd1a680>], 'test': [<function sample_sin_fnc at 0x20000b76eef0>, <function sample_linear_fnc at 0x2000afd1a680>], 'valid': [<function sample_sin_fnc at 0x20000b76eef0>, <function sample_linear_fnc at 0x2000afd1a680>]}


ipdb>  q


BdbQuit: 

In [2]:
args = get_args("")
args.log_interval=50

args.task = 'sine' # Level-1 task
args.k_batch_train = [10, 3]
args.k_batch_test =  [10, 3]
args.k_batch_valid = [10, 3]
args.n_batch_train = [10, 3]
args.n_batch_test = [10, 3]
args.n_batch_valid = [10, 3]
args.architecture = [ 1, 28, 1 ]
args.n_iters = [4, 20 ] 
args.for_iters = [ 1,1 ]
args.n_contexts = [ 3 ]
args.lrs = [ 1.0,  0.001] 
args.test_interval = 100 
args.classes = [0]

# args.log_name=

args.ctx_logging_levels=[0,]

print(args)

main(args)


Namespace(architecture=[1, 28, 1], classes=[0], ctx_logging_levels=[0], data_parallel=False, device=device(type='cuda', index=0), encoders=[None, None, None], first_order=False, for_iters=[1, 1], higher_flag=False, k_batch_test=[10, 3], k_batch_train=[10, 3], k_batch_valid=[10, 3], load_model='', log_interval=50, log_name=None, lrs=[1.0, 0.001], model_type='CAVIA', n_batch_test=[10, 3], n_batch_train=[10, 3], n_batch_valid=[10, 3], n_contexts=[3], n_iters=[4, 20], prefix='', save_path='/nobackup/users/benhuh/Projects/cavia/shared_results', seed=42, task='sine', test_interval=100, v_num=None, viz=False)
sine
/nobackup/users/benhuh/Projects/cavia/shared_results/logs/sine
max_iters [4, 20]
level 1
batch 3
task <function sample_sin_fnc at 0x200099fbb0e0>
level 1 type train
> /nobackup/users/benhuh/Projects/cavia/regression/dataset.py(156)sample_meta_lv_data()
    154             elif callable(task):
    155                 set_trace()
--> 156                 subtask_list = [task(sample_typ

ipdb>  p task


<function sample_sin_fnc at 0x200099fbb0e0>


ipdb>  cont


level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304c560>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304c560>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304c3b0>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304c3b0>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d440>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <fun

...

level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d7a0>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d7a0>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d830>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d830>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304d9e0>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <fun

ipdb>  p task


<function sample_sin_fnc at 0x200099fbb0e0>


ipdb>  cont


level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304dc20>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304dc20>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304db00>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304db00>)
level 0
batch 10
task (<function regression_input_function at 0x200099fbb170>, <function get_sin_function.<locals>.sin_function at 0x2000a304da70>)
level 0 type train
level 0 type test
level 0 type valid
Task_loader Level 0 task (<function regression_input_function at 0x200099fbb170>, <fun